In [1]:
def read_and_process_file(filename='input'):
    file1 = open(f'{filename}_15', 'r')
    Lines = [l.rstrip() for l in file1.readlines()]
    res = []
    for l in Lines:
        _sensor, _beacon = l.split(": ")
        _sensor = [int(v) for v in _sensor.replace("Sensor at x=","").replace("y=", "").split(", ")]
        _beacon = [int(v) for v in _beacon.replace("closest beacon is at x=","").replace("y=", "").split(", ")]
        res.append((_sensor,_beacon))
    return res

#### One star

In [2]:
def find_unavailable_spots(dataset='input', row=2000000):
    selected_row = row
    positions = []
    beacons = []
    data = read_and_process_file(dataset)

    for s in data:
        if s[1][1]==selected_row:
            beacons.append(s[1][0])
        sensor_range = abs(s[0][0]-s[1][0])+abs(s[0][1]-s[1][1])
        distance_to_row = abs(s[0][1]-selected_row)
        if distance_to_row>sensor_range:
            #print(f"Sensor doesn't reach row {selected_row}")
            pass
        else:
            row_range = sensor_range - distance_to_row
            for i in range(row_range+1):
                #print(f"Adding {i} for {s}")
                positions.append(s[0][0]+i)
                positions.append(s[0][0]-i)            

    res = list(set(positions) - set(beacons))
    print(f"Number of positions in row {selected_row} that beacon cannot be present is: {len(res)}")
    return res

In [8]:
x = find_unavailable_spots('test',10)

Number of positions in row 10 that beacon cannot be present is: 26


#### Two stars

In [4]:
def tuning_freq(x,y):
    return x*4000000 + y

In [5]:
def merge(intervals):
    intervals.sort(key=lambda x: x[0])
    merged = []
    for interval in intervals:
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
            merged[-1][1] = max(merged[-1][1], interval[1])
    return merged

In [6]:
from collections import defaultdict
def find_distress_beacon(dataset='input', search_range = 4000000):
    data = read_and_process_file(dataset)
    
    busy_ranges = defaultdict(list)
    for s in data:
        sensor_range = abs(s[0][0]-s[1][0])+abs(s[0][1]-s[1][1])
        for r in range(sensor_range+1):
            c = sensor_range-r
            _low = s[0][0]-r
            _high = s[0][0]+r
            if _low>=0:
                busy_ranges[_low].append([max(s[0][1]-c,0),min(s[0][1]+c, search_range)+1])
            if _high<=search_range:
                busy_ranges[_high].append([max(s[0][1]-c,0),min(s[0][1]+c, search_range)+1])
    
    for _row, _ranges in busy_ranges.items():
        _ranges = merge(_ranges)
        if len(_ranges)>1:
            _col = _ranges[0][1]
            print(f"The distress beacon is at: {_row} - {_col}. Tuning frequency: {tuning_freq(_row, _col)}")
            return

In [7]:
find_distress_beacon(dataset='input', search_range = 4000000)

The distress beacon is at: 3335216 - 3187704. Tuning frequency: 13340867187704
